In [45]:
import spacy
# import pytextrank
import textacy.ke.textrank

import glob
import pandas as pd
import collections
import re, random

In [3]:
files = glob.glob('./incoPat_225509/*.xls')

dfs = []
for f in files:
    try:
        dfs.append(pd.read_excel(f))
    except Exception as e:
        print("{0}: {1}".format(f, e))

df = pd.concat(dfs)
df = df.reset_index(drop=True)

In [32]:
text = df['标题(英)'].str.cat(sep=' . ')

In [65]:
# load a spaCy model, depending on language, scale, etc.
nlp = spacy.load("en_core_web_sm")
nlp.max_length = 15600000

# add PyTextRank to the spaCy pipeline
# tr = pytextrank.TextRank()
# nlp.add_pipe(tr.PipelineComponent, name="textrank", last=True)

offset = random.randint(1, 10000000)
doc = nlp(text[offset:offset+5000000], disable = ['ner'])
sorted_dict = dict(textacy.ke.textrank(doc, topn=3000))

In [66]:
# # examine the top-ranked phrases in the document
# sorted_dict = {}
# for p in doc._.phrases:
# #     print("{:.4f} {:5d}  {}".format(p.rank, p.count, p.text))
#     sorted_dict[p.text] = p.rank
# #     print(p.chunks)

In [67]:
filter_dict = collections.OrderedDict()
re_ = r"^A |^THE |^THEIR |^ITS |^THIS |^AN |^SUCH A |^AS |^TO |^AND |^-PRON- |AT LEAST|USE THEREOF|^OR | THEREOF| THEREFOR| THERETO| THEREFROM| THEREBY"
for k, v in sorted_dict.items():
    k = re.sub(re_, "", k.upper())
    if  k.count(' ') < 4 and  k.count(' ') >= 1 and k.count('(') == 0 and k.count(')') == 0:
        filter_dict[k] = v
filter_df = pd.DataFrame.from_dict(filter_dict, orient='index', columns=['score'])

In [69]:
filter_df.head(1000).to_excel('./out/kws_textrank.xlsx')

In [64]:
len(sorted_dict)

1000